In [ ]:
# Perform Synthesizer Parameter Search

In [1]:
import IPython.display as ipd

from ohc.search.search import EvolutionarySearch
from ohc.search.search import FindMySound
from ohc.vst import VSTHost
from ohc.fitness.fitness import FitnessFunction
from ohc.fitness.clap import CLAPSimilarity

%load_ext autoreload
%autoreload 2

/Users/svanka/miniforge3/envs/ohc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-25 15:01:05,823	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [19]:
#check if gpu is available
import torch
print("GPU Available: ", torch.cuda.is_available()) 

GPU Available:  False


In [5]:
PLUGIN_PATH = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"
SAMPLE_RATE = 48000
BLOCK_SIZE = 512

In [6]:
# Create a VST
vst = VSTHost(PLUGIN_PATH, "fixed", SAMPLE_RATE, BLOCK_SIZE)

# Select parameters to search over
params = vst.list_params(filter_midi_cc=True)
active_params = [p for p in params if p != "Bypass"]
vst.set_active_params(active_params)

error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/Dexed.vst3'
2024-02-25 15:01:37,253	INFO worker.py:1724 -- Started a local Ray instance.


In [7]:
# Initialize the CLAP embedding
clap = CLAPSimilarity(sample_rate=SAMPLE_RATE, device = 'cpu')

Initializing CLAP model: laion/larger_clap_general
Initializing CLAP tokenizer: laion/larger_clap_general
Initializing CLAP feature extractor: laion/larger_clap_general


In [13]:
# Create a fitness function
text_target = ["snoring in sleep"]
fitness = FitnessFunction(vst, clap, text_target=text_target, clap_batch_size=128)

In [14]:
problem = FindMySound(
    "max",
    num_params=len(active_params),
    min_value=0.0,
    max_value=1.0,
    fitness_function=fitness,
)

[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The `dtype` for the problem's decision variables is set as torch.float32
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The `device` of the problem is set as cpu
[2024-02-25 15:04:08] INFO     <85875> evotorch.core: Instance of `FindMySound` (id:14276837680) -- The number of actors that will be allocated for parallelized evaluation is 0


In [17]:
search = EvolutionarySearch(problem, population_size=100, max_generations=50, logger="custom")

In [18]:
search.run()

Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips
Processing batch of 100 audio clips


KeyboardInterrupt: 

In [ ]:
best_discovered_solution = search._searcher.status["best"]

In [ ]:
audio, _ = vst.render_now(best_discovered_solution.values.unsqueeze(0), 48, 1.0, 1.0)[0]

In [ ]:
ipd.Audio(audio, rate=SAMPLE_RATE)